In [1]:
import pandas as pd
from datasets import load_dataset

In [2]:
from google.colab import files
uploaded = files.upload()

Saving NER dataset.csv to NER dataset.csv


In [3]:
import pandas as pd

In [5]:
import pandas as pd

# Try with Windows-1252 first
df = pd.read_csv("NER dataset.csv", encoding="windows-1252")
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [6]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

In [7]:
df = df.drop(columns=["POS"])

In [8]:
df["Sentence #"] = df["Sentence #"].fillna(method="ffill")

/tmp/ipython-input-3199379459.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["Sentence #"] = df["Sentence #"].fillna(method="ffill")


In [14]:
# ===== Group into sentences =====
sentences = []
labels = []

# Fill any potential NaN values in 'Word' with empty strings before grouping
df["Word"] = df["Word"].fillna("")

for _, group in df.groupby("Sentence #"):
    words = group["Word"].tolist()
    tags = group["Tag"].tolist()
    sentences.append(words)
    labels.append(tags)

In [10]:
# Create label list
label_list = sorted(list(set(tag for doc in labels for tag in doc)))
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

In [15]:
# ===== Prepare dict for HuggingFace Dataset =====
data_dict = {
    "tokens": sentences,  # list of lists of strings
    "ner_tags": [[label_to_id[tag] for tag in doc] for doc in labels]  # list of lists of ints
}

# Ensure all types are correct before creating dataset
assert all(isinstance(tok, str) for doc in data_dict["tokens"] for tok in doc)
assert all(isinstance(tag, int) for doc in data_dict["ner_tags"] for tag in doc)

# ===== Convert to Dataset =====
dataset = Dataset.from_dict(data_dict)

# ===== Train-test split =====
train_test = dataset.train_test_split(test_size=0.1, seed=42)
dataset = DatasetDict({
    "train": train_test["train"],
    "test": train_test["test"]
})

print(dataset)
print("Labels:", label_list)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 43163
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 4796
    })
})
Labels: ['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per', 'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org', 'I-per', 'I-tim', 'O']


In [18]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=e08cce32987ee46fa501feabf6253dca3373ec6bd5aced2f87ab631032cd840b
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [19]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
from seqeval.metrics import classification_report
import numpy as np

In [20]:
# =============================
# 4. Tokenizer
# =============================
model_checkpoint = "bert-base-cased"  # You can change to any compatible model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Align labels with tokens
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Ignore special tokens
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)  # Ignore sub-tokens
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/43163 [00:00<?, ? examples/s]

Map:   0%|          | 0/4796 [00:00<?, ? examples/s]

In [21]:
# =============================
# 5. Model
# =============================
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# =============================
# 6. Data Collator
# =============================
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


In [23]:
# =============================
# 7. Metrics
# =============================
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [id_to_label[l] for l in label if l != -100]
        for label in labels
    ]
    true_predictions = [
        [id_to_label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    report = classification_report(true_labels, true_predictions, digits=4)
    print(report)

    return {
        "f1": float(report.split()[-2]),  # last F1 score in report
    }


In [27]:
# =============================
# 8. Training Arguments
# =============================
args = TrainingArguments(
    "ner-model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    report_to="none"
)


In [ ]:
# =============================
# 9. Trainer
# =============================
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# =============================
# 10. Train
# =============================
trainer.train()

/tmp/ipython-input-1219450972.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [ ]:
# =============================
# 11. Evaluate
# =============================
trainer.evaluate()

In [ ]:
from transformers import pipeline

checkpoint = ""
token_classifier = pipeline(
    "token-classification", model=checkpoint, aggregation_strategy="simple"
)

token_classifier("")